In [ ]:
function get_ab_phase(co, step, V0, wid, H_Kin, U_0, Degeneracy)
    A = 1
    U_list = []
    for x_t in x0:step:x1
        imps = [ impurity(V0, [x_t,y0], wid) ]
        H1 = Himp(imps, H_Kin)
        E, U = eigenstates(dense(H1))
        push!(U_list, U)
        A *= (U_0[1:Degeneracy]'*U[1:Degeneracy]) / abs(U_0[1:Degeneracy]'*U[1:Degeneracy])
        U_0 = copy(U)
    end
    for y_t in y1:step:y2
        imps = [ impurity(V0, [x1,y_t], wid)]
        H1 = Himp(imps, H_Kin)
        E, U = eigenstates(dense(H1))
        push!(U_list, U)
        A *= (U_0[1:Degeneracy]'*U[1:Degeneracy]) / abs(U_0[1:Degeneracy]'*U[1:Degeneracy])
        U_0 = copy(U)
    end
    for x_t in x2:-step:x3
        imps = [ impurity(V0, [x_t,y2], wid)]
        H1 = Himp(imps, H_Kin)
        E, U = eigenstates(dense(H1))
        push!(U_list, U)
        A *= (U_0[1:Degeneracy]'*U[1:Degeneracy]) / abs(U_0[1:Degeneracy]'*U[1:Degeneracy])
        U_0 = copy(U)
    end
    for y_t in y3:-step:y0
        imps = [ impurity(V0, [x3,y_t], wid)]
        H1 = Himp(imps, H_Kin)
        E, U = eigenstates(dense(H1))
        push!(U_list, U)
        A *= (U_0[1:Degeneracy]'*U[1:Degeneracy]) / abs(U_0[1:Degeneracy]'*U[1:Degeneracy])
        U_0 = copy(U)
    end
    AB_Phase = angle(A)/pi
    return AB_Phase, U_list
end

In [ ]:
function get_ab_phase2(coord, step, V0, wid, N, co, sp_basis, H1, U_0, Degeneracy)
    A = 1
    U_list = []
    for x_t in x0:step:x1
        imps = [ impurity(V0, [x_t,y0], wid) ]
        H1 = Himp(imps, N, co, sp_basis, H1)
        E, U = eigenstates(dense(H1))
        push!(U_list, U)
        A *= (U_0[1:Degeneracy]'*U[1:Degeneracy]) / abs(U_0[1:Degeneracy]'*U[1:Degeneracy])
        U_0 = copy(U)
    end
    for y_t in y1:step:y2
        imps = [ impurity(V0, [x1,y_t], wid)]
        H1 = Himp(imps, N, co, sp_basis, H1)
        E, U = eigenstates(dense(H1))
        push!(U_list, U)
        A *= (U_0[1:Degeneracy]'*U[1:Degeneracy]) / abs(U_0[1:Degeneracy]'*U[1:Degeneracy])
        U_0 = copy(U)
    end
    for x_t in x2:-step:x3
        imps = [ impurity(V0, [x_t,y2], wid)]
        H1 = Himp(imps, N, co, sp_basis, H1)
        E, U = eigenstates(dense(H1))
        push!(U_list, U)
        A *= (U_0[1:Degeneracy]'*U[1:Degeneracy]) / abs(U_0[1:Degeneracy]'*U[1:Degeneracy])
        U_0 = copy(U)
    end
    for y_t in y3:-step:y0
        imps = [ impurity(V0, [x3,y_t], wid)]
        H1 = Himp(imps, N, co, sp_basis, H1)
        E, U = eigenstates(dense(H1))
        push!(U_list, U)
        A *= (U_0[1:Degeneracy]'*U[1:Degeneracy]) / abs(U_0[1:Degeneracy]'*U[1:Degeneracy])
        U_0 = copy(U)
    end
    AB_Phase = angle(A)/pi
    return AB_Phase, U_list
end

In [ ]:
function get_ab_phase_projection(co, step, V0, wid, H_Kin, U_0, Degeneracy, P1M, P1_dagM, P2M, P2_dagM, basis_tilde, basis_mb_tilde, Int_Operator, basis_tilde2)
    A = 1
    U_list = []
    for x_t in x0:step:x1
        imps = [ impurity(V0, [x_t,y0], wid) ]
        H1 = Himp_proj(imps, H_Kin, P1M, P1_dagM, P2M, P2_dagM, basis_tilde, basis_mb_tilde, Int_Operator, basis_tilde2)
        E, U = eigenstates(dense(H1))
        push!(U_list, U)
        A *= (U_0[1:Degeneracy]'*U[1:Degeneracy]) / abs(U_0[1:Degeneracy]'*U[1:Degeneracy])
        U_0 = copy(U)
    end
    for y_t in y1:step:y2
        imps = [ impurity(V0, [x1,y_t], wid)]
        H1 = Himp_proj(imps, H_Kin, P1M, P1_dagM, P2M, P2_dagM, basis_tilde, basis_mb_tilde, Int_Operator, basis_tilde2)
        E, U = eigenstates(dense(H1))
        push!(U_list, U)
        A *= (U_0[1:Degeneracy]'*U[1:Degeneracy]) / abs(U_0[1:Degeneracy]'*U[1:Degeneracy])
        U_0 = copy(U)
    end
    for x_t in x2:-step:x3
        imps = [ impurity(V0, [x_t,y2], wid)]
        H1 = Himp_proj(imps, H_Kin, P1M, P1_dagM, P2M, P2_dagM, basis_tilde, basis_mb_tilde, Int_Operator, basis_tilde2)
        E, U = eigenstates(dense(H1))
        push!(U_list, U)
        A *= (U_0[1:Degeneracy]'*U[1:Degeneracy]) / abs(U_0[1:Degeneracy]'*U[1:Degeneracy])
        U_0 = copy(U)
    end
    for y_t in y3:-step:y0
        imps = [ impurity(V0, [x3,y_t], wid)]
        H1 = Himp_proj(imps, H_Kin, P1M, P1_dagM, P2M, P2_dagM, basis_tilde, basis_mb_tilde, Int_Operator, basis_tilde2)
        E, U = eigenstates(dense(H1))
        push!(U_list, U)
        A *= (U_0[1:Degeneracy]'*U[1:Degeneracy]) / abs(U_0[1:Degeneracy]'*U[1:Degeneracy])
        U_0 = copy(U)
    end
    AB_Phase = angle(A)/pi
    return AB_Phase, U_list
end

In [3]:
function projection_operators(cut_off, basis, U_0, PN)
    substates = 1:cut_off
    
    basis_tilde = SubspaceBasis(basis, U_0[substates])
    basis_tilde2 = basis_tilde⊗basis_tilde
    
    states_mb_tilde = bosonstates(basis_tilde, PN)
    basis_mb_tilde  = ManyBodyBasis(basis_tilde, states_mb_tilde)
    
    P1 = projector(basis, basis_tilde)
    P1_dag = dagger(P1)
    P1M = P1.data
    P1_dagM = P1_dag.data
    
    P2M = (P1⊗P1).data
    P2_dagM = (P1_dag⊗P1_dag).data
    
    return P1, P1_dag, P1M, P1_dagM, P2M, P2_dagM, basis_tilde, basis_tilde2, basis_mb_tilde
end

projection_operators (generic function with 1 method)

In [4]:
function Himp_proj(imps, H_Kin, P1M, P1_dagM, P2M, P2_dagM, basis_tilde, basis_mb_tilde, Int_Operator, basis_tilde2)

    Total_Ham = Himp(imps, H_Kin)

    H1M = Total_Ham.data
    H1_tildeM = P1_dagM * H1M * P1M
    H1_tilde = Operator(basis_tilde, H1_tildeM)
    H1_tilde = (H1_tilde + H1_tilde')/2
    H_NI_tilde_mb = manybodyoperator(basis_mb_tilde, H1_tilde) 
    
    V2M = Int_Operator.data
    V2_tildeM =  P2_dagM * V2M * P2M
    V2_tilde = Operator(basis_tilde2, V2_tildeM)
    Vint_tilde_mb = manybodyoperator(basis_mb_tilde, V2_tilde)

    H_tilde_mb = H_NI_tilde_mb + Vint_tilde_mb
    H_tilde_mb = (H_tilde_mb + H_tilde_mb')/2
    
    return H_tilde_mb
end

Himp_proj (generic function with 1 method)

In [ ]:
function get_Braiding_Phase_NI(Step_B, V0, w0, H_Kin, UB_0, Degeneracy)
    A = 1
    U_list = []
    for i in 1:Step_B:2
        θ = i * pi
        imp_pos = circular_path(θ,center,radius)
        imp_pos2 = circular_path(θ+pi,center,radius)
        imps = [ impurity(1.1*V0, imp_pos2, w0), impurity(V0, imp_pos, w0) ]
        H1 = Himp(imps, H_Kin)
        E, U = eigenstates(dense(H1))
        push!(U_list, U)
        A *= (UB_0[1:Degeneracy]'*U[1:Degeneracy]) / abs(UB_0[1:Degeneracy]'*U[1:Degeneracy])
        UB_0 = copy(U)
    end
    AB_Phase = angle(A)/pi
    return AB_Phase, U_list
end

In [1]:
function get_Braiding_Phase_proj(Step_B, V0, w0, H_Kin, UB_0, Degeneracy, P1M, P1_dagM, P2M, P2_dagM, basis_tilde, basis_mb_tilde, Int_Operator, basis_tilde2)
    A = 1
    U_list = []
    for i in 1:Step_B:2
        θ = i * pi
        imp_pos = circular_path(θ,center,radius)
        imp_pos2 = circular_path(θ+pi,center,radius)
        imps = [ impurity(1.1*V0, imp_pos2, w0), impurity(V0, imp_pos, w0) ]
        H1 = Himp_proj(imps, H_Kin, P1M, P1_dagM, P2M, P2_dagM, basis_tilde, basis_mb_tilde, Int_Operator, basis_tilde2)
        E, U = eigenstates(dense(H1))
        push!(U_list, U)
        A *= (UB_0[1:Degeneracy]'*U[1:Degeneracy]) / abs(UB_0[1:Degeneracy]'*U[1:Degeneracy])
        U_0 = copy(U)
    end
    AB_Phase = angle(A)/pi
    return AB_Phase, U_list
end

get_Braiding_Phase_proj (generic function with 1 method)

In [ ]:
function get_ab_phase_rectangular_2_pot(co, step, V0, wid, H_Kin, U_0, Degeneracy)
    A = 1
    U_list = []
    for (x_t1, x_t2) in zip(x0:step:x1, x2:-step:x3)
        imps = [ impurity(V0, [x_t1,y0], wid)
                 impurity(V0, [x_t2,y2], wid) ]
        H1 = Himp(imps, H_Kin)
        E, U = eigen(dense(H1).data) #!!!!!
        push!(U_list, U)
        A *= (U_0[1:Degeneracy]'*U[1:Degeneracy]) / abs(U_0[1:Degeneracy]'*U[1:Degeneracy])
        U_0 = copy(U)
    end
    for (y_t1, y_t2) in zip(y1:step:y2, y3:-step:y0)
        imps = [ impurity(V0, [x1,y_t1], wid)
                 impurity(V0, [x3,y_t2], wid) ]
        H1 = Himp(imps, H_Kin)
        E, U = eigen(dense(H1).data)
        push!(U_list, U)
        A *= (U_0[1:Degeneracy]'*U[1:Degeneracy]) / abs(U_0[1:Degeneracy]'*U[1:Degeneracy])
        U_0 = copy(U)
    end
    for (x_t1, x_t2) in zip(x2:-step:x3, x0:step:x1)
        imps = [ impurity(V0, [x_t1,y2], wid)
                 impurity(V0, [x_t2,y0], wid) ]
        H1 = Himp(imps, H_Kin)
        E, U = eigen(dense(H1).data)
        push!(U_list, U)
        A *= (U_0[1:Degeneracy]'*U[1:Degeneracy]) / abs(U_0[1:Degeneracy]'*U[1:Degeneracy])
        U_0 = copy(U)
    end
    for (y_t1, y_t2) in zip(y3:-step:y0, y1:step:y2)
        imps = [ impurity(V0, [x3,y_t1], wid)
                 impurity(V0, [x1,y_t2], wid) ]
        H1 = Himp(imps, H_Kin)
        E, U = eigen(dense(H1).data)
        push!(U_list, U)
        A *= (U_0[1:Degeneracy]'*U[1:Degeneracy]) / abs(U_0[1:Degeneracy]'*U[1:Degeneracy])
        U_0 = copy(U)
    end
    AB_Phase = angle(A)/pi
    return AB_Phase, U_list
end